In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
import numpy as np
import sys
import import_ipynb
import quickplay


Using TensorFlow backend.


importing Jupyter notebook from quickplay.ipynb


In [2]:
x=quickplay.Simulation()
print("1 "+str(x.getBuyState(x.p1)))
print("2 "+str(x.getBuyState(x.p2)))
wins = [0]*10
for i in range(len(wins)):
    j=0
    while True:
        j+=1
        print(str(j)+" p1: "+str(x.makeDecision(x.p1,1)))
        if x.shop.checkEnd():
            print("shop end")
            print(x.shop.cards)
            break
        print(str(j)+" p2: "+str(x.makeDecision(x.p2,1)))
        if x.shop.checkEnd():
            print("shop end")
            print(x.shop.cards)
            break
    print(x.p1.getVP())
    print(x.p2.getVP())
    if x.p1.getVP() > x.p2.getVP():
        print(str(x.p1)+" wins!")
        wins[i]=1
    else:
        print(str(x.p2)+" wins!")
        wins[i]=2
    x=quickplay.Simulation()
print(str(wins))

Game Started
Copper purchased by Player 1
Estate purchased by Player 1
Copper purchased by Player 2
Estate purchased by Player 2
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 3, 0, 0, 0, 0]
Player 2 wins!
1 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 3, 0, 0, 1, 2]
2 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 3, 0, 0, 1, 3]
purchase options: ['Cellar', 'Moat', 'Estate']
2
1
{'Cellar': 10, 'Market': 10, 'Militia': 10, 'Mine': 10, 'Moat': 10, 'Remodel': 10, 'Smithy': 10, 'Village': 10, 'Woodcutter': 10, 'Workshop': 10, 'Copper': 46, 'Silver': 40, 'Gold': 30, 'Estate': 8, 'Dutchy': 8, 'Province': 8}
False
Moat purchased by Player 1
1 p1: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 7, 0, 0, 3, 0, 0, 1, 5]
purchase options: ['Cellar', 'Moat', 'Village', 'Woodcutter', 'Workshop', 'Silver', 'Estate']
3
1
{'Cellar': 10, 'Market': 10, 'Militia': 10, 'Mine': 10, 'Moat': 9, 'Remodel': 10, 'Smithy': 10, 'Village': 10, 'Woodcutter': 10, 'Workshop': 10, 'Copper': 46, 'Silver': 40, 'Gold': 30, 'Estate': 8, 'Dutchy': 8, 'Provi

purchase options: ['Market', 'Militia', 'Mine', 'Remodel', 'Smithy', 'Dutchy']
5
1
{'Cellar': 9, 'Market': 9, 'Militia': 10, 'Mine': 10, 'Moat': 7, 'Remodel': 7, 'Smithy': 9, 'Village': 9, 'Woodcutter': 8, 'Workshop': 9, 'Copper': 42, 'Silver': 39, 'Gold': 30, 'Estate': 8, 'Dutchy': 8, 'Province': 8}
False
Remodel purchased by Player 2
random action choice of Moat
Moat played by Player 2
9 p2: [0, 0, 0, 0, 2, 3, 1, 1, 1, 0, 8, 0, 0, 3, 0, 0, 1, 3]
purchase options: ['Copper']
1
1
{'Cellar': 9, 'Market': 9, 'Militia': 10, 'Mine': 10, 'Moat': 7, 'Remodel': 6, 'Smithy': 9, 'Village': 9, 'Woodcutter': 8, 'Workshop': 9, 'Copper': 42, 'Silver': 39, 'Gold': 30, 'Estate': 8, 'Dutchy': 8, 'Province': 8}
False
Copper purchased by Player 1
random action choice of Moat
Moat played by Player 1
10 p1: [1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 11, 1, 0, 3, 0, 0, 1, 3]
purchase options: ['Cellar', 'Moat', 'Village', 'Woodcutter', 'Workshop', 'Silver', 'Estate']
3
1
{'Cellar': 9, 'Market': 9, 'Militia': 10, 'Mine

Cellar played by Player 2
random action choice of Smithy
Smithy played by Player 2
11 p2: [3, 0, 0, 1, 0, 0, 2, 0, 0, 0, 5, 4, 2, 4, 0, 0, 1, 2]
purchase options: ['Copper']
1
1
{'Cellar': 6, 'Market': 9, 'Militia': 9, 'Mine': 9, 'Moat': 9, 'Remodel': 9, 'Smithy': 8, 'Village': 10, 'Woodcutter': 9, 'Workshop': 7, 'Copper': 44, 'Silver': 37, 'Gold': 29, 'Estate': 4, 'Dutchy': 7, 'Province': 8}
False
Copper purchased by Player 1
random action choice of Workshop
Workshop played by Player 1
Woodcutter purchased by Player 1
12 p1: [1, 1, 1, 0, 1, 1, 0, 0, 2, 3, 10, 0, 0, 6, 1, 0, 1, 2]
purchase options: ['Cellar', 'Moat', 'Estate']
2
1
{'Cellar': 6, 'Market': 9, 'Militia': 9, 'Mine': 9, 'Moat': 9, 'Remodel': 9, 'Smithy': 8, 'Village': 10, 'Woodcutter': 8, 'Workshop': 7, 'Copper': 43, 'Silver': 37, 'Gold': 29, 'Estate': 4, 'Dutchy': 7, 'Province': 8}
False
Moat purchased by Player 2
random action choice of Smithy
Smithy played by Player 2
12 p2: [3, 0, 0, 1, 1, 0, 2, 0, 0, 0, 5, 4, 2, 4, 0, 

random action choice of Cellar
Cellar played by Player 2
random action choice of Militia
Militia played by Player 2
2 p2: [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 3, 0, 0, 1, 5]
purchase options: ['Militia', 'Remodel', 'Smithy', 'Village', 'Woodcutter', 'Workshop', 'Silver']
4
1
{'Cellar': 9, 'Market': 10, 'Militia': 8, 'Mine': 10, 'Moat': 10, 'Remodel': 10, 'Smithy': 10, 'Village': 10, 'Woodcutter': 9, 'Workshop': 10, 'Copper': 46, 'Silver': 40, 'Gold': 30, 'Estate': 8, 'Dutchy': 8, 'Province': 8}
False
Silver purchased by Player 1
3 p1: [0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 7, 1, 0, 3, 0, 0, 1, 3]
purchase options: ['Market', 'Militia', 'Mine', 'Remodel', 'Smithy', 'Dutchy']
5
1
{'Cellar': 9, 'Market': 10, 'Militia': 8, 'Mine': 10, 'Moat': 10, 'Remodel': 10, 'Smithy': 10, 'Village': 10, 'Woodcutter': 9, 'Workshop': 10, 'Copper': 46, 'Silver': 39, 'Gold': 30, 'Estate': 8, 'Dutchy': 8, 'Province': 8}
False
Dutchy purchased by Player 2
3 p2: [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 3, 1, 0, 1, 2

random action choice of Cellar
Cellar played by Player 1
random action choice of Remodel
Remodel played by Player 1
Copper purchased by Player 1
24 p1: [3, 0, 2, 0, 2, 1, 2, 0, 2, 4, 16, 2, 0, 7, 2, 0, 1, 4]
purchase options: ['Copper']
1
1
{'Cellar': 2, 'Market': 10, 'Militia': 7, 'Mine': 10, 'Moat': 5, 'Remodel': 7, 'Smithy': 8, 'Village': 7, 'Woodcutter': 5, 'Workshop': 1, 'Copper': 31, 'Silver': 38, 'Gold': 30, 'Estate': 0, 'Dutchy': 6, 'Province': 8}
False
Copper purchased by Player 2
random action choice of Cellar
Cellar played by Player 2
24 p2: [5, 0, 1, 0, 3, 2, 0, 3, 3, 5, 14, 0, 0, 7, 0, 0, 1, 4]
purchase options: ['Militia', 'Remodel', 'Smithy', 'Village', 'Woodcutter', 'Workshop', 'Silver']
4
1
{'Cellar': 2, 'Market': 10, 'Militia': 7, 'Mine': 10, 'Moat': 5, 'Remodel': 7, 'Smithy': 8, 'Village': 7, 'Woodcutter': 5, 'Workshop': 1, 'Copper': 30, 'Silver': 38, 'Gold': 30, 'Estate': 0, 'Dutchy': 6, 'Province': 8}
False
Woodcutter purchased by Player 1
random action choice of S

{'Cellar': 4, 'Market': 5, 'Militia': 7, 'Mine': 9, 'Moat': 5, 'Remodel': 7, 'Smithy': 3, 'Village': 5, 'Woodcutter': 8, 'Workshop': 8, 'Copper': 35, 'Silver': 34, 'Gold': 29, 'Estate': 3, 'Dutchy': 6, 'Province': 7}
False
Estate purchased by Player 2
random action choice of Cellar
Cellar played by Player 2
random action choice of Cellar
Cellar played by Player 2
29 p2: [2, 3, 1, 1, 4, 2, 4, 1, 1, 1, 11, 4, 1, 4, 1, 1, 1, 0]
purchase options: ['Copper']
0
1
{'Cellar': 4, 'Market': 5, 'Militia': 7, 'Mine': 9, 'Moat': 5, 'Remodel': 7, 'Smithy': 3, 'Village': 5, 'Woodcutter': 8, 'Workshop': 8, 'Copper': 35, 'Silver': 34, 'Gold': 29, 'Estate': 2, 'Dutchy': 6, 'Province': 7}
False
Copper purchased by Player 1
random action choice of Village
Village played by Player 1
random action choice of Village
Village played by Player 1
random action choice of Village
Village played by Player 1
30 p1: [4, 2, 2, 0, 1, 1, 3, 4, 1, 1, 14, 3, 0, 8, 1, 0, 1, 4]
purchase options: ['Copper']
0
1
{'Cellar': 4,

In [53]:
#print(wins.count(1))
#print(wins.count(2))
print(x.shop.cards)
print(len(list(filter(lambda y: x.shop.cards[y]==0 , x.shop.cards)))>=3 or x.shop.cards["Province"]==0 )
print(x.shop.checkEnd())

{'Cellar': 0, 'Market': 4, 'Militia': 3, 'Mine': 10, 'Moat': 0, 'Remodel': 0, 'Smithy': 3, 'Village': 0, 'Woodcutter': 4, 'Workshop': 0, 'Copper': 17, 'Silver': 33, 'Gold': 26, 'Estate': 0, 'Dutchy': 7, 'Province': 6}
True
False


In [18]:
model = Sequential()
model.add(Dense(10, activation='sigmoid', input_shape=(18,)))
model.add(Dense(2, activation='linear'))
model.compile(loss='mse', optimizer='adam', metrics=['mae'])

In [19]:
y = 0.95
eps = 0.45
decay_factor = 0.999
r_avg_list = []
num_episodes = 50
for i in range(num_episodes):
    s = env.reset()
    eps *= decay_factor
    if i % 5 == 0:
        print("Episode {} of {}".format(i + 1, num_episodes))
    done = False
    r_sum = 0
    while not done:
        if np.random.random() < eps:
            a = np.random.randint(0, 2)
        else:
            a = np.argmax(model.predict(np.identity(5)[s:s + 1]))
        new_s, r, done, _ = env.step(a)
        target = r + y * np.max(model.predict(np.identity(5)[new_s:new_s + 1]))
        target_vec = model.predict(np.identity(5)[s:s + 1])[0]
        target_vec[a] = target
        model.fit(np.identity(5)[s:s + 1], target_vec.reshape(-1, 2), epochs=1, verbose=0)
        s = new_s
        r_sum += r
    r_avg_list.append(r_sum / 1000)

NameError: name 'env' is not defined